In [1]:
import tensorflow as tf
import keras
import keras_tuner as kt
import sklearn
from sklearn.model_selection import train_test_split
import os

In [2]:
# cwd = os.getcwd()
# os.mkdir(os.path.join(cwd, 'mlops'))

In [3]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

In [4]:
X_train = x_train.reshape(-1,28, 28, 1) 
X_test = x_test.reshape(-1,28,28,1)
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

In [5]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2)
print(X_train.shape)

(48000, 28, 28, 1)


In [6]:
class DeepTuner(kt.Tuner):
    def run_trial(self, trial, X, y, validation_data, **fit_kwargs):
        model = self.hypermodel.build(trial.hyperparameters)
        model.fit(X, y, batch_size=trial.hyperparameters.Choice(
            'batch_size', [16, 32]), **fit_kwargs)


        X_val, y_val = validation_data
        eval_scores = model.evaluate(X_val, y_val)
        return {name: value for name, value in zip(
            model.metrics_names,
            eval_scores)}

In [7]:
def build_model(hp):
    model = tf.keras.Sequential()
    model.add(tf.keras.Input(shape = X_train.shape[1:], name = 'inputs'))
    for i in range(hp.Int('num_layers', min_value=1, max_value=10)):
              model.add(tf.keras.layers.Conv2D(hp.Int(
                  'units_{i}'.format(i=i), min_value=32, max_value=128, step=5), (3,3),activation='relu'))
    model.add(tf.keras.layers.Flatten())
    for i in range(hp.Int('n_connections', 1, 3)):
        model.add(tf.keras.layers.Dense(hp.Choice(f'n_nodes',
                                  values=[32,64,128, 256]), activation = 'relu'))
    model.add(tf.keras.layers.Dense(10, activation='softmax', name = 'outputs'))
    model.compile(optimizer = 'adam',loss='categorical_crossentropy',
        metrics=['accuracy'])
    return model

In [8]:
my_keras_tuner = DeepTuner(
    oracle=kt.oracles.BayesianOptimizationOracle(
        objective=kt.Objective('accuracy', 'max'),
        max_trials=10,
        seed=42),
    hypermodel=build_model,
    overwrite=True,
    project_name='my_keras_tuner')

# 해당 모델 학습시간은 약 10분정도 걸립니다!
my_keras_tuner.search(
    X_train, y_train, validation_data=(X_val, y_val), epochs=3)

Trial 10 Complete [00h 00m 48s]
accuracy: 0.9835000038146973

Best accuracy So Far: 0.9850000143051147
Total elapsed time: 00h 08m 07s


In [9]:
best_hps = my_keras_tuner.get_best_hyperparameters(num_trials=10)[0]
model = build_model(best_hps)
model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_65 (Conv2D)          (None, 26, 26, 32)        320       
                                                                 
 conv2d_66 (Conv2D)          (None, 24, 24, 32)        9248      
                                                                 
 conv2d_67 (Conv2D)          (None, 22, 22, 32)        9248      
                                                                 
 conv2d_68 (Conv2D)          (None, 20, 20, 32)        9248      
                                                                 
 conv2d_69 (Conv2D)          (None, 18, 18, 32)        9248      
                                                                 
 conv2d_70 (Conv2D)          (None, 16, 16, 32)        9248      
                                                                 
 conv2d_71 (Conv2D)          (None, 14, 14, 32)      

In [10]:
model.fit(X_train, y_train, batch_size=32, epochs = 5)

Epoch 1/5
1500/1500 [==============================] - 9s 6ms/step - loss: 0.1780 - accuracy: 0.9450
Epoch 2/5
1500/1500 [==============================] - 9s 6ms/step - loss: 0.0669 - accuracy: 0.9807
Epoch 3/5
1500/1500 [==============================] - 9s 6ms/step - loss: 0.0528 - accuracy: 0.9831
Epoch 4/5
1500/1500 [==============================] - 9s 6ms/step - loss: 0.0390 - accuracy: 0.9885
Epoch 5/5
1500/1500 [==============================] - 9s 6ms/step - loss: 0.0328 - accuracy: 0.9901


In [11]:
model.evaluate(X_test, y_test)

313/313 [==============================] - 1s 3ms/step - loss: 0.0540 - accuracy: 0.9845


[0.053998615592718124, 0.984499990940094]

In [12]:
cwd = os.getcwd()
folder_path = os.path.join(cwd, 'mlops')
# os.mkdir(os.path.join(folder_path, 'best_model'))
folder_path_s = os.path.join(folder_path, 'best_model')
# os.mkdir(os.path.join(folder_path_s, '1'))
save_path = os.path.join(folder_path_s, '1')
fname = os.path.join(save_path, 'model')
model.save(fname)

INFO:tensorflow:Assets written to: C:\Users\박해극\Aiffel\aiffel_quest_re\DL_lms\exploration6\mlops\best_model\1\model\assets


INFO:tensorflow:Assets written to: C:\Users\박해극\Aiffel\aiffel_quest_re\DL_lms\exploration6\mlops\best_model\1\model\assets


In [13]:
!docker pull tensorflow/serving

Using default tag: latest


error during connect: this error may indicate that the docker daemon is not running: Post "http://%2F%2F.%2Fpipe%2Fdocker_engine/v1.24/images/create?fromImage=tensorflow%2Fserving&tag=latest": open //./pipe/docker_engine: The system cannot find the file specified.


In [15]:
!docker run -p 8500:8500 \
			-p 8501:8501 \
			--mount type=bind, source=/tmp/models, target=/models/my_model -e MODEL_NAME=saved_model.pb \
			-e MODEL_BASE_PATH=/Aiffel/aiffel_quest_re/DL_lms/exploration6/mlops/best_model/1/model/saved_model.pb \
			-t tensorflow/serving

invalid argument "type=bind," for "--mount" flag: invalid field '' must be a key=value pair
See 'docker run --help'.
